In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Jul 26 19:34:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    44W /  70W |   6172MiB / 15109MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.5 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys

In [ ]:
# #restaurant no segment phase A
train_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseA/Restaurant/main/Res_train_no_token_change_icon.csv'
dev_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseA/Restaurant/main/Res_dev_no_token_change_icon.csv'
test_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseA/Restaurant/main/Res_test_no_token_change_icon.csv'

In [ ]:
#hotel no segment phase A
# train_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseA/Hotel/main/Hos_train_no_sentence_segment_1.csv'
# dev_path =  '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseA/Hotel/main/Hos_dev_no_token_no_icon_1.csv'
# test_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseA/Hotel/main/Hos_test_no_token_no_icon_1.csv'

In [ ]:
#Hotel phase B
# train_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Hos/Hos_train_no_segment_PhaseB.csv'
# dev_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Hos/Hos_dev_no_PhaseB.csv'
# test_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Hos/Hos_test_PhaseB.csv'

In [ ]:
# #Res Phase B
# train_path  = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Res/Res_train_no_segment_phaseB.csv'
# dev_path =  '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Res/Res_dev_phaseB.csv'
# test_path = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Res/Res_test_phaseB.csv'

In [ ]:
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

In [ ]:
print(len(train.columns))
print(len(dev.columns))
print(len(test.columns))

14
14
14


In [ ]:
train.columns

Index(['id', 'content', 'FOOD#STYLE&OPTIONS', 'FOOD#QUALITY',
       'AMBIENCE#GENERAL', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL',
       'FOOD#PRICES', 'RESTAURANT#PRICES', 'LOCATION#GENERAL',
       'RESTAURANT#MISCELLANEOUS', 'DRINKS#STYLE&OPTIONS', 'DRINKS#PRICES',
       'DRINKS#QUALITY'],
      dtype='object')

In [ ]:
#restaurant phase A
target_list = ['FOOD#STYLE&OPTIONS', 'FOOD#QUALITY',
       'AMBIENCE#GENERAL', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL',
       'FOOD#PRICES', 'RESTAURANT#PRICES', 'LOCATION#GENERAL',
       'RESTAURANT#MISCELLANEOUS', 'DRINKS#STYLE&OPTIONS', 'DRINKS#PRICES',
       'DRINKS#QUALITY']

In [ ]:
# hotel phase A
# target_list = ['HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL',
#        'LOCATION#GENERAL', 'SERVICE#GENERAL', 'HOTEL#COMFORT',
#        'HOTEL#CLEANLINESS', 'FACILITIES#GENERAL', 'ROOMS#CLEANLINESS',
#        'ROOM_AMENITIES#COMFORT', 'ROOMS#COMFORT', 'FACILITIES#PRICES',
#        'ROOM_AMENITIES#GENERAL', 'FOOD&DRINKS#STYLE&OPTIONS', 'ROOMS#PRICES',
#        'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 'HOTEL#QUALITY',
#        'ROOMS#QUALITY', 'FOOD&DRINKS#QUALITY', 'HOTEL#PRICES',
#        'FACILITIES#DESIGN&FEATURES', 'FACILITIES#QUALITY',
#        'ROOM_AMENITIES#QUALITY', 'ROOM_AMENITIES#CLEANLINESS',
#        'ROOM_AMENITIES#DESIGN&FEATURES', 'HOTEL#MISCELLANEOUS',
#        'FOOD&DRINKS#PRICES', 'FACILITIES#COMFORT', 'FOOD&DRINKS#MISCELLANEOUS',
#        'FACILITIES#CLEANLINESS', 'FACILITIES#MISCELLANEOUS',
#        'ROOMS#MISCELLANEOUS', 'ROOM_AMENITIES#MISCELLANEOUS',
#        'ROOM_AMENITIES#PRICES']
# len(target_list)

In [ ]:
#hotel Phase B
# target_list = ['FACILITIES#CLEANLINESS positive',
#        'FACILITIES#COMFORT positive', 'FACILITIES#DESIGN&FEATURES negative',
#        'FACILITIES#DESIGN&FEATURES positive', 'FACILITIES#GENERAL positive',
#        'FACILITIES#QUALITY negative', 'FACILITIES#QUALITY positive',
#        'FOOD&DRINKS#QUALITY negative', 'FOOD&DRINKS#QUALITY positive',
#        'FOOD&DRINKS#STYLE&OPTIONS negative',
#        'FOOD&DRINKS#STYLE&OPTIONS positive', 'HOTEL#CLEANLINESS positive',
#        'HOTEL#COMFORT negative', 'HOTEL#COMFORT positive',
#        'HOTEL#DESIGN&FEATURES negative', 'HOTEL#DESIGN&FEATURES positive',
#        'HOTEL#GENERAL negative', 'HOTEL#GENERAL neutral',
#        'HOTEL#GENERAL positive', 'HOTEL#MISCELLANEOUS negative',
#        'HOTEL#PRICES negative', 'HOTEL#PRICES positive',
#        'HOTEL#QUALITY negative', 'HOTEL#QUALITY positive',
#        'LOCATION#GENERAL negative', 'LOCATION#GENERAL positive',
#        'ROOMS#CLEANLINESS negative', 'ROOMS#CLEANLINESS positive',
#        'ROOMS#COMFORT negative', 'ROOMS#COMFORT positive',
#        'ROOMS#DESIGN&FEATURES negative', 'ROOMS#DESIGN&FEATURES positive',
#        'ROOMS#GENERAL positive', 'ROOMS#PRICES positive',
#        'ROOMS#QUALITY positive', 'ROOM_AMENITIES#DESIGN&FEATURES negative',
#        'ROOM_AMENITIES#DESIGN&FEATURES positive',
#        'ROOM_AMENITIES#GENERAL positive', 'ROOM_AMENITIES#QUALITY negative',
#        'SERVICE#GENERAL negative', 'SERVICE#GENERAL neutral',
#        'SERVICE#GENERAL positive']
# len(target_list)

In [ ]:
# # Res Phase B
# target_list = ['FOOD#STYLE&OPTIONS#neutral', 'FOOD#QUALITY#neutral',
#        'AMBIENCE#GENERAL#neutral', 'RESTAURANT#GENERAL#neutral',
#        'SERVICE#GENERAL#neutral', 'FOOD#PRICES#neutral',
#        'RESTAURANT#PRICES#neutral', 'LOCATION#GENERAL#neutral',
#        'RESTAURANT#MISCELLANEOUS#neutral', 'DRINKS#STYLE&OPTIONS#neutral',
#        'DRINKS#PRICES#neutral', 'DRINKS#QUALITY#neutral',
#        'FOOD#STYLE&OPTIONS#negative', 'FOOD#QUALITY#negative',
#        'AMBIENCE#GENERAL#negative', 'RESTAURANT#GENERAL#negative',
#        'SERVICE#GENERAL#negative', 'FOOD#PRICES#negative',
#        'RESTAURANT#PRICES#negative', 'LOCATION#GENERAL#negative',
#        'RESTAURANT#MISCELLANEOUS#negative', 'DRINKS#STYLE&OPTIONS#negative',
#        'DRINKS#PRICES#negative', 'DRINKS#QUALITY#negative',
#        'FOOD#STYLE&OPTIONS#positive', 'FOOD#QUALITY#positive',
#        'AMBIENCE#GENERAL#positive', 'RESTAURANT#GENERAL#positive',
#        'SERVICE#GENERAL#positive', 'FOOD#PRICES#positive',
#        'RESTAURANT#PRICES#positive', 'LOCATION#GENERAL#positive',
#        'RESTAURANT#MISCELLANEOUS#positive', 'DRINKS#STYLE&OPTIONS#positive',
#        'DRINKS#PRICES#positive', 'DRINKS#QUALITY#positive']
# print(len(target_list))

In [ ]:
#res phaseB
# target_list = ['FOOD#STYLE&OPTIONS#neutral', 'FOOD#QUALITY#neutral',
#        'AMBIENCE#GENERAL#neutral', 'RESTAURANT#GENERAL#neutral',
#        'SERVICE#GENERAL#neutral', 'FOOD#PRICES#neutral',
#        'RESTAURANT#PRICES#neutral', 'LOCATION#GENERAL#neutral',
#        'RESTAURANT#MISCELLANEOUS#neutral', 'DRINKS#STYLE&OPTIONS#neutral',
#        'DRINKS#PRICES#neutral', 'DRINKS#QUALITY#neutral',
#        'FOOD#STYLE&OPTIONS#negative', 'FOOD#QUALITY#negative',
#        'AMBIENCE#GENERAL#negative', 'RESTAURANT#GENERAL#negative',
#        'SERVICE#GENERAL#negative', 'FOOD#PRICES#negative',
#        'RESTAURANT#PRICES#negative', 'LOCATION#GENERAL#negative',
#        'RESTAURANT#MISCELLANEOUS#negative', 'DRINKS#STYLE&OPTIONS#negative',
#        'DRINKS#PRICES#negative', 'DRINKS#QUALITY#negative',
#        'FOOD#STYLE&OPTIONS#positive', 'FOOD#QUALITY#positive',
#        'AMBIENCE#GENERAL#positive', 'RESTAURANT#GENERAL#positive',
#        'SERVICE#GENERAL#positive', 'FOOD#PRICES#positive',
#        'RESTAURANT#PRICES#positive', 'LOCATION#GENERAL#positive',
#        'RESTAURANT#MISCELLANEOUS#positive', 'DRINKS#STYLE&OPTIONS#positive',
#        'DRINKS#PRICES#positive', 'DRINKS#QUALITY#positive']

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-05
NUM_LABELS = len(target_list)

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer, max_len):
    self.df = df
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.title = self.df['content']
    self.targets = self.df[target_list].values

  def __len__(self):
    return len(self.title)
    
  def __getitem__(self, index):
    title = str(self.title[index])
    title = " ".join(title.split())

    inputs = self.tokenizer.encode_plus(
        title,
        None,
        add_special_tokens = True,
        max_length = self.max_len,
        padding = 'max_length',
        return_token_type_ids = True,
        truncation=True,
        return_attention_mask = True,
        return_tensors = 'pt'
    )
    return {
        'input_ids':inputs['input_ids'].flatten(),
        'attention_mask': inputs['attention_mask'].flatten(),
        'token_type_ids': inputs['token_type_ids'].flatten(),
        'targets': torch.FloatTensor(self.targets[index])
    }

In [ ]:
train_dataset = CustomDataset(train, tokenizer, MAX_LEN)
val_dataset = CustomDataset(dev, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle = True,
    batch_size = TRAIN_BATCH_SIZE,
    num_workers = 0
)
val_data_loader = torch.utils.data.DataLoader(
    val_dataset,
    shuffle = False,
    batch_size = VALID_BATCH_SIZE,
    num_workers = 0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
  checkpoint = torch.load(checkpoint_fpath)
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  valid_loss_min = checkpoint['valid_loss_min']
  return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
  f_path = checkpoint_path
  torch.save(state, f_path)
  if is_best:
    best_fpath = best_model_path
    shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(nn.Module):
  def __init__(self):
    super(BERTClass, self).__init__()
    # self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased', return_dict=True)
    self.bert_model = BertModel.from_pretrained('vinai/phobert-base', return_dict=True)
    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768, NUM_LABELS) 
  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert_model(input_ids, attention_mask, token_type_ids)
    output_dropout = self.dropout(output.pooler_output)
    output = self.linear(output_dropout)
    return output

In [ ]:
model = BERTClass()
model.to(device)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing BertModel: ['roberta.encoder.layer.9.attention.output.LayerNorm.weight', 'roberta.encoder.layer.8.output.LayerNorm.bias', 'roberta.encoder.layer.6.output.LayerNorm.bias', 'roberta.encoder.layer.5.output.dense.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.2.attention.output.dense.bias', 'roberta.encoder.layer.8.attention.self.value.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.7.attention.self.query.weight', 'roberta.encoder.layer.7.attention.self.key.weight', 'roberta.pooler.dense.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.11.attention.output.LayerNorm.weight', 'lm_head.b

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tr

In [ ]:
def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [ ]:
def multi_label_metrics(y_pred, y_true):
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

In [ ]:
def get_output(outputs):
  final_output = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
  t = np.array(final_output)
  THRESHOLD = 0.5
  upper, lower = 1, 0
  final_out = np.where(t > THRESHOLD, upper, lower)
  return final_out.tolist()
def get_labels(labels):
  targets = labels.cpu().detach().numpy().tolist()
  return targets

In [ ]:
def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model,
    optimizer,
    checkpoint_path,
    best_model_path
):
  train_loss_set = []
  val_loss_set = []
  valid_loss_min = np.Inf
  for epoch in range(1, n_epochs + 1):
    print('Epoch {}/{}'.format(epoch, n_epochs))
    train_loss = 0
    valid_loss = 0
    model.train()
    #training loop
    output_train = []
    target_train = []
    for index, batch in enumerate(train_data_loader):
      input_ids = batch['input_ids'].to(device, dtype = torch.long)
      attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
      targets = batch['targets'].to(device, dtype = torch.float)
      outputs = model(input_ids, attention_mask, token_type_ids)
      # print(outputs)
      for output in get_output(outputs):
        output_train.append(output)
      for target in get_labels(targets):
        target_train.append(target)
      # print(multi_label_metrics(outputs, targets, threshold=0.5))
      optimizer.zero_grad()
      loss = loss_fn( outputs, targets)
      # train_loss_set.append(loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss = train_loss + (1/(index+1)*(loss.item()-train_loss))
    #validation loop
    model.eval()
    with torch.no_grad():
      for index, batch in enumerate(train_data_loader):
        input_ids = batch['input_ids'].to(device, dtype = torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        valid_loss = valid_loss + (1/(index+1)*(loss.item()-train_loss))
    checkpoint = {
        'epoch': epoch+1,
        'valid_loss_min': valid_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
    train_loss_set.append(train_loss)
    val_loss_set.append(valid_loss)
    print(multi_label_metrics(output_train, target_train))
    print('Loss: {}, Val_Loss:{}'.format(train_loss, valid_loss))
  return model

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader,  val_data_loader, model, optimizer, "/curr_ckpt", "/best.pt" )

Epoch 1/10
{'f1': 0.5977671053452254, 'roc_auc': 0.7206609165678665, 'accuracy': 0.12094594594594595}
Loss: 0.4126785913462281, Val_Loss:-0.14438821135777705
Epoch 2/10
{'f1': 0.6397159586395914, 'roc_auc': 0.7452491260255876, 'accuracy': 0.17804054054054055}
Loss: 0.37853922510659826, Val_Loss:-0.049667022081207726
Epoch 3/10
{'f1': 0.6617181796057434, 'roc_auc': 0.7600402043981732, 'accuracy': 0.19324324324324324}
Loss: 0.3667831831080938, Val_Loss:-0.1273724332316239
Epoch 4/10
{'f1': 0.710661699485542, 'roc_auc': 0.792582723030184, 'accuracy': 0.24695945945945946}
Loss: 0.3413807086406214, Val_Loss:-0.0595739718327536
Epoch 5/10
{'f1': 0.7361863173216886, 'roc_auc': 0.8103521796591053, 'accuracy': 0.2760135135135135}
Loss: 0.3204641962243664, Val_Loss:-0.009121839811417105
Epoch 6/10
{'f1': 0.7507080515577135, 'roc_auc': 0.8204805297193306, 'accuracy': 0.2939189189189189}
Loss: 0.30847350003257884, Val_Loss:-0.11671626631520539
Epoch 7/10
{'f1': 0.7589773053720195, 'roc_auc': 0.826

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/save model/phoBertResPhaseA.pt')

In [ ]:
# testing
def get_predict(review):
  example = review
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  trained_model.eval()

  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = trained_model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      t = np.array(final_output)
      THRESHOLD = 0.35
      upper, lower = 1, 0

      final_out = np.where(t > THRESHOLD, upper, lower)
  return final_out.tolist()

In [ ]:
dev_predicts = []
for x in dev['content'].values:
  dev_predicts.append(get_predict(x)[0])

In [ ]:
test_predicts = []
for x in test['content'].values:
  test_predicts.append(get_predict(x)[0])

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [ ]:
print("Val")
print(classification_report(dev[target_list].values, dev_predicts, target_names = target_list))

Val
                          precision    recall  f1-score   support

      FOOD#STYLE&OPTIONS       0.57      0.83      0.68       516
            FOOD#QUALITY       0.90      1.00      0.95      1159
        AMBIENCE#GENERAL       0.70      0.76      0.73       205
      RESTAURANT#GENERAL       0.29      0.74      0.42       231
         SERVICE#GENERAL       0.66      0.88      0.75       228
             FOOD#PRICES       0.92      0.94      0.93       724
       RESTAURANT#PRICES       0.00      0.00      0.00        35
        LOCATION#GENERAL       0.29      0.43      0.34       134
RESTAURANT#MISCELLANEOUS       0.00      0.00      0.00       117
    DRINKS#STYLE&OPTIONS       0.00      0.00      0.00        10
           DRINKS#PRICES       0.00      0.00      0.00        44
          DRINKS#QUALITY       0.00      0.00      0.00        39

               micro avg       0.70      0.83      0.76      3442
               macro avg       0.36      0.46      0.40      3442
    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("Test")
print(classification_report(test[target_list].values, test_predicts, target_names = target_list))

Test
                          precision    recall  f1-score   support

      FOOD#STYLE&OPTIONS       0.87      0.90      0.88       403
            FOOD#QUALITY       0.92      1.00      0.96       457
        AMBIENCE#GENERAL       0.75      0.73      0.74       254
      RESTAURANT#GENERAL       0.47      0.95      0.62       222
         SERVICE#GENERAL       0.47      0.91      0.62       175
             FOOD#PRICES       0.79      0.88      0.83       331
       RESTAURANT#PRICES       0.67      0.03      0.05        73
        LOCATION#GENERAL       0.49      0.89      0.63       179
RESTAURANT#MISCELLANEOUS       0.00      0.00      0.00       130
    DRINKS#STYLE&OPTIONS       0.00      0.00      0.00        45
           DRINKS#PRICES       0.00      0.00      0.00        76
          DRINKS#QUALITY       0.00      0.00      0.00        70

               micro avg       0.69      0.76      0.72      2415
               macro avg       0.45      0.52      0.45      2415
   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
